# Training

In [1]:
import os
from PIL import Image
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


import numpy as np

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None # Turn off some annoying pandas warning

In [2]:
# Setting seed for reproducible results

SEED = 42

tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

In [3]:
import os
import cv2
import numpy as np

# Define the folder paths
folder_path1 = r'G:\Deep Learning in Computer Vision (DMET1067)\Milestone 2\Actors DataSet predifined\Actors DataSet\Karim Abdelaziz\Training'
folder_path2 = r'G:\Deep Learning in Computer Vision (DMET1067)\Milestone 2\Actors DataSet predifined\Actors DataSet\Karim Abdelaziz\Testing'
folder_path3 = r'G:\Deep Learning in Computer Vision (DMET1067)\Milestone 2\Actors DataSet predifined\Actors DataSet\Karim Abdelaziz\Validation'

folder_path4 = r'G:\Deep Learning in Computer Vision (DMET1067)\Milestone 2\Actors DataSet predifined\Actors DataSet\Hend Sabry\Training'
folder_path5 = r'G:\Deep Learning in Computer Vision (DMET1067)\Milestone 2\Actors DataSet predifined\Actors DataSet\Hend Sabry\Testing'
folder_path6 = r'G:\Deep Learning in Computer Vision (DMET1067)\Milestone 2\Actors DataSet predifined\Actors DataSet\Hend Sabry\Testing'

folder_path7 = r'G:\Deep Learning in Computer Vision (DMET1067)\Milestone 2\Actors DataSet predifined\Actors DataSet\Hany Ramzy\Training'
folder_path8 = r'G:\Deep Learning in Computer Vision (DMET1067)\Milestone 2\Actors DataSet predifined\Actors DataSet\Hany Ramzy\Testing'
folder_path9 = r'G:\Deep Learning in Computer Vision (DMET1067)\Milestone 2\Actors DataSet predifined\Actors DataSet\Hany Ramzy\Validation'

# Function to read images from a folder and resize them
def read_images_from_folder(folder_path, label):
    images = []
    labels = []

    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        try:
                # Open image using PIL
                img = Image.open(img_path)

                # Convert to RGB mode if not already in that mode
                if img.mode != 'RGB':
                    img = img.convert('RGB')

                images.append(img)
                labels.append(label)
        except Exception as e:
                print(f"Error loading image {image_path}: {e}")

        if img is not None:
            images.append(img)
            labels.append(label)

    return images, labels

# Paths to your image folders and labels
folder_paths = [folder_path1, folder_path2, folder_path3, folder_path4, folder_path5, folder_path6, folder_path7, folder_path8, folder_path9]
labels = ["Karim AbdelAziz", "Karim AbdelAziz", "Karim AbdelAziz", "Hend Sabry", "Hend Sabry", "Hend Sabry", "Hany Ramzy", "Hany Ramzy", "Hany Ramzy"]  # Example labels for each folder

# Initialize empty lists to store images and corresponding labels
all_images = []
all_labels = []

# Read images from each folder and store them in all_images list
# Also, store corresponding labels in all_labels list
for folder_path, label in zip(folder_paths, labels):
    images, label_list = read_images_from_folder(folder_path, label)
    all_images.extend(images)
    all_labels.extend(label_list)

# Convert lists to numpy arrays
all_images = np.array(all_images)
all_labels = np.array(all_labels)

# Save the images and labels arrays to numpy files
np.save("images.npy", all_images)
np.save("labels.npy", all_labels)

print("Images and labels saved successfully!")


c:\Users\super\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: FutureWarning: The input object of type 'PngImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PngImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
c:\Users\super\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Images and labels saved successfully!


In [4]:
print(all_images[5000])
print(all_labels[5000])

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x512 at 0x28CC290AB70>
Hany Ramzy


In [5]:
df = pd.DataFrame({"image": all_images, "label": all_labels})

In [6]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
df_train.label.value_counts()

Karim AbdelAziz    2390
Hany Ramzy         1574
Hend Sabry          826
Name: label, dtype: int64

In [8]:
unique_labels = df['label'].unique()
print("Unique labels in the DataFrame:")
print(unique_labels)

Unique labels in the DataFrame:
['Karim AbdelAziz' 'Hend Sabry' 'Hany Ramzy']


In [9]:
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}
print("label2id mapping:")
print(label2id)
print("id2label mapping:")
print(id2label)

label2id mapping:
{'Karim AbdelAziz': 0, 'Hend Sabry': 1, 'Hany Ramzy': 2}
id2label mapping:
{0: 'Karim AbdelAziz', 1: 'Hend Sabry', 2: 'Hany Ramzy'}


In [10]:
# Map labels to ids

df_train['class'] = df_train['label'].map(label2id)
df_test['class'] = df_test['label'].map(label2id)


df_train = df_train.sample(frac=1, random_state=SEED)
df_test = df_test.sample(frac=1, random_state=SEED)

In [11]:
def fix_image(img):
  
  rgb_img = img.convert("RGB")

  return np.array(rgb_img)

In [12]:
train_images = np.array([fix_image(img) for img in df_train['image'].to_list()])
test_images = np.array([fix_image(img) for img in df_test['image'].to_list()])

In [13]:
train_labels = np.array([label for label in df_train['class'].to_list()])
test_labels = np.array([label for label in df_test['class'].to_list()])

In [14]:
train_labels = to_categorical(train_labels, 3)
test_labels = to_categorical(test_labels, 3)

In [15]:
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((4790, 512, 512, 3), (4790, 3), (1198, 512, 512, 3), (1198, 3))

In [ ]:
np.save('train_images.npy', train_images)
np.save('test_images.npy', test_images)
np.save('train_labels.npy', train_labels)
np.save('test_labels.npy', test_labels)

In [ ]:
train_images = np.load('train_images.npy')
test_images = np.load('test_images.npy')
train_labels = np.load('train_labels.npy')
test_labels = np.load('test_labels.npy')

In [16]:
num_labels = 3

In [19]:

# Convert labels to one-hot encoding
num_classes = 3

model = Sequential()

# Feature Extraction Stage
model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="valid", activation='relu', input_shape=(512, 512, 3))) # Number of parameters = 8 x (3 x 3) x 3 + 8 = 224
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="valid", activation='relu', input_shape=(512, 512, 3))) # Number of parameters = 8 x (3 x 3) x 3 + 8 = 224
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="valid", activation='relu')) # Number of parameters = 16 x (5 x 5) x 8 + 16 = 3216
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), padding="valid", activation='relu', input_shape=(512, 512, 3))) # Number of parameters = 8 x (3 x 3) x 3 + 8 = 224
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(filters=16, kernel_size=(7, 7), strides=(1, 1), padding="valid", activation='relu')) # Number of parameters = 32 x (3 x 3) x 16 + 32 = 4640
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# Classification Stage
model.add(Dense(units=128, activation='relu')) # Should be sigmoid but sigmoid has decresed the accuracy thats why we used relu
model.add(Dropout(0.5))
model.add(Dense(num_labels, activation='softmax')) # Number of parameters = 256 x 6 + 6 = 15426 = 1542
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x=train_images, y=train_labels, validation_split=0.2, batch_size=32, epochs=5, shuffle=True)

# Save the trained model
model.save('trained_model.h5')


Epoch 1/5
120/120 [==============================] - 439s 4s/step - loss: 1.0490 - accuracy: 0.5598 - val_loss: 0.6282 - val_accuracy: 0.7296
Epoch 2/5
120/120 [==============================] - 440s 4s/step - loss: 0.6197 - accuracy: 0.7343 - val_loss: 0.3581 - val_accuracy: 0.8800
Epoch 3/5
120/120 [==============================] - 437s 4s/step - loss: 0.2859 - accuracy: 0.8972 - val_loss: 0.2574 - val_accuracy: 0.9081
Epoch 4/5
120/120 [==============================] - 436s 4s/step - loss: 0.2008 - accuracy: 0.9175 - val_loss: 0.1910 - val_accuracy: 0.9186
Epoch 5/5
120/120 [==============================] - 435s 4s/step - loss: 0.2016 - accuracy: 0.9165 - val_loss: 0.2538 - val_accuracy: 0.9050


In [20]:
model.save('trained_model.h5')

In [21]:
test_new_labels = model.predict(test_images)


38/38 [==============================] - 30s 780ms/step


In [24]:
from sklearn.metrics import accuracy_score

# Convert one-hot encoded test_labels back to categorical labels
test_labels_categorical = np.argmax(test_labels, axis=1)

# Convert one-hot encoded test_new_labels to categorical labels
test_new_labels_categorical = np.argmax(test_new_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(test_labels_categorical, test_new_labels_categorical)

# Print accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 90.73%
